## ВКР: обработка данных

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

На этом этапе мы предобработаем наши данные, собранные на этапе `scraping.ipynb`.

По итогу мы получим признаки:
- ррр
- ррр
See Project Proposal.

Где библиотеки??? See `scraping.ipynb`

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit
import re
import numpy as np

In [2]:
import nltk
# nltk.download('stopwords')
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
morph = MorphAnalyzer()

### Пост-обработка

Импорт данных:

In [10]:
df = pd.read_csv('resumes_features.csv')  #.sample(3000)
# df = pd.read_csv('backup/16.11/resumes_features.csv').sample(3000)  # <- после 18.04

In [11]:
df.drop(list(df)[0], axis=1, inplace=True)
df.reset_index(inplace=True)
df.drop(list(df)[0], axis=1, inplace=True)

In [12]:
df

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,...,Релокация?,Коммандировка?,Город?,Опыт нормализованный,Уровень оборазования нормализованный,Карьерный рост,"О себе, summarized","О себе, tokens","О себе, avg_tokens","О себе, 3-grams"
0,https://hh.ru/resume/a179dc8e00074bc7400039ed1...,Администратор салона красоты,"Москва, м. Молодежная, не готова к переезду, г...",Опыт работы 7 лет 1 месяц,"Уверенный пользователь ПК, Ms Word, Ms Excel, ...",Высшее образование\n2015\nФГБУ ВПО «Российский...,Специализации:\nАдминистратор\nЗанятость: полн...,Коммуникации с клиентами и партнерами/Работа с...,ФГБУ ВПО «Российский экономический университет...,ФГБУ ВПО «Саратовская Государственная Юридичес...,...,готова к редким командировкам,не готова к переезду,Москва,---,Высшее образование,False,"Уверенный пользователь ПК, Ms Word, Ms Excel, ...","[array([-0.00977549, 0.00884461, -0.00854385,...",-0.000048,"['уверенный пользователь пк', 'пользователь пк..."
1,https://hh.ru/resume/cdaf5cfe00003bcc320039ed1...,Руководитель отдела планирования и управления ...,"Москва, м. Выхино, готова к переезду, готова к...",Опыт работы 17 лет 3 месяца,"Энергична, активна, в поиске новых возможносте...",Высшее образование\n2010\nМосковский Государст...,Специализации:\nНачальник склада\nЗанятость: п...,---,Московский Государственный Университет Экономи...,NaN,...,готова к командировкам,готова к переезду,Москва,---,Высшее образование,False,В настоящее время занимаюсь благотворительной ...,"[array([ 9.1362959e-03, -5.2859355e-03, -8.334...",0.000095,"['настоящий время заниматься', 'время занимать..."
2,https://hh.ru/resume/fb6ac9c200027770d60039ed1...,Начальник отдела продаж,"Шахты, готова к переезду (Москва, Санкт-Петерб...",Опыт работы 18 лет 4 месяца,"опыт прямых продаж, ведение коммерческих перег...",Высшее образование\n2001\nЮжно-Российский госу...,Специализации:\nРуководитель отдела продаж\nЗа...,---,Южно-Российский государственный университет эк...,NaN,...,готова к командировкам,готова к переезду,Шахты,---,Высшее образование,False,"В крупнейшем медучреждении в России, который з...","[array([ 6.8795979e-03, 5.5447384e-03, 4.711...",0.000058,"['крупный медучреждение россия', 'медучреждени..."
3,https://hh.ru/resume/e23c95770000b1b95f0039ed1...,"Коммерческий директор, Финансовый директор","Москва, м. Бабушкинская, готов к переезду, гот...",Опыт работы 17 лет 5 месяцев,Коммуникабелен\nУмение работать с возражениями...,Высшее образование\n2006\nБрянский государстве...,Специализации:\nКоммерческий директор (CCO)\nФ...,---,Брянский государственный технический университет,Брянский государственный технический университет,...,готов к командировкам,готов к переезду,Москва,---,Высшее образование,True,В настоящее время занимаюсь инвестиционной дея...,"[array([ 9.1362959e-03, -5.2859355e-03, -8.334...",-0.000263,"['настоящий время заниматься', 'время занимать..."
4,https://hh.ru/resume/9c86655b00073d88700039ed1...,QA/auto QA/Performance QA,"Краснодар, готов к переезду (Москва, Санкт-Пет...",Опыт работы 7 лет 10 месяцев,Дисциплина - умение выполнять поставленные зад...,Среднее специальное образование\n2009\nКраснод...,Специализации:\nТестировщик\nЗанятость: стажир...,---,Краснодарский Государственный Гуманитарно Техн...,NaN,...,готов к командировкам,готов к переезду,Краснодар,---,Среднее специальное образование,True,"В настоящее время работаю в компании, которая ...","[array([-9.12124198e-03, 1.25383922e-05, -8.8...",0.000357,"['настоящий время работать', 'время работать к..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,https://hh.ru/resume/b74645f3000102b1b90039ed1...,Финансовый аналитик,"Москва, м. Крылатское, не готова к переезду, н...",Опыт работы 8 лет 11 месяцев,"Уверенный пользователь ПК: MS Office (Word, Ex...",Высшее образование\n2009\nЧувашский государств...,"Специализации:\nФинансовый аналитик, инвестици...",---,Чувашский государственный университет имени И....,NaN

**Перевод опыта работы в числовое значение:**

В данной функции мы приводим к единому виду опыт работы. были найдены: years, year, months, etc.

In [5]:
#df.drop(list(df)[0], axis=1, inplace=True)
#df.drop(list(df)[0], axis=1, inplace=True)
#df.reset_index(inplace=True)
#df.drop(list(df)[0], axis=1, inplace=True)

In [6]:
df

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,...,Кол-во образований,Работа 1,Работа 2,Топовость образования,is_eng?,Справочник по интересам,Навыки списком,Релокация?,Коммандировка?,Город?
0,https://hh.ru/resume/a179dc8e00074bc7400039ed1...,Администратор салона красоты,"Москва, м. Молодежная, не готова к переезду, г...",Опыт работы 7 лет 1 месяц,"Уверенный пользователь ПК, Ms Word, Ms Excel, ...",Высшее образование\n2015\nФГБУ ВПО «Российский...,Специализации:\nАдминистратор\nЗанятость: полн...,Коммуникации с клиентами и партнерами/Работа с...,ФГБУ ВПО «Российский экономический университет...,ФГБУ ВПО «Саратовская Государственная Юридичес...,...,2,Салон красоты SHOKO\nУслуги для населения... П...,Кредитный Потребительский Кооператив «Поволжск...,False,ENG,---,['Коммуникации с клиентами и партнерами/Работа...,готова к редким командировкам,не готова к переезду,Москва
1,https://hh.ru/resume/cdaf5cfe00003bcc320039ed1...,Руководитель отдела планирования и управления ...,"Москва, м. Выхино, готова к переезду, готова к...",Опыт работы 17 лет 3 месяца,"Энергична, активна, в поиске новых возможносте...",Высшее образование\n2010\nМосковский Государст...,Специализации:\nНачальник склада\nЗанятость: п...,---,Московский Государственный Университет Экономи...,NaN,...,1,"АО ТД Перекресток\nМосква, karusel.ru/\nРознич...","Дочки & Сыночки, сеть супермаркетов\nРоссия, d...",False,RU,---,['---'],готова к командировкам,готова к переезду,Москва
2,https://hh.ru/resume/fb6ac9c200027770d60039ed1...,Начальник отдела продаж,"Шахты, готова к переезду (Москва, Санкт-Петерб...",Опыт работы 18 лет 4 месяца,"опыт прямых продаж, ведение коммерческих перег...",Высшее образование\n2001\nЮжно-Российский госу...,Специализации:\nРуководитель отдела продаж\nЗа...,---,Южно-Российский государственный университет эк...,NaN,...,1,Государственное бюджетное учреждение Ростовско...,"Государственное бюджетное учреждение ""Лечебно-...",False,RU,---,['---'],готова к командировкам,готова к переезду,Шахты
3,https://hh.ru/resume/e23c95770000b1b95f0039ed1...,"Коммерческий директор, Финансовый директор","Москва, м. Бабушкинская, готов к переезду, гот...",Опыт работы 17 лет 5 месяцев,Коммуникабелен\nУмение работать с возражениями...,Высшее образование\n2006\nБрянский государстве...,Специализации:\nКоммерческий директор (CCO)\nФ...,---,Брянский государственный технический университет,Брянский государственный технический университет,...,2,"Альфа-Банк (Россия)\nМосква, www.alfabank.ru\n...","Банк ВТБ (ПАО)\nМосква, www.vtbcareer.com\nФин...",False,RU,---,['---'],готов к командировкам,готов к переезду,Москва
4,https://hh.ru/resume/9c86655b00073d88700039ed1...,QA/auto QA/Performance QA,"Краснодар, готов к переезду (Москва, Санкт-Пет...",Опыт работы 7 лет 10 месяцев,Дисциплина - умение выполнять поставленные зад...,Среднее специальное образование\n2009\nКраснод...,Специализации:\nТестировщик\nЗанятость: стажир...,---,Краснодарский Государственный Гуманитарно Техн...,NaN,...,1,"JavaRush\nМосква, javarush.ru/\nИнформационные...","Social Links\nМосква, mtg-bi.com\nИнформационн...",False,ENG,---,['---'],готов к командировкам,готов к переезду,Краснодар
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,https://hh.ru/resume/b74645f3000102b1b90039ed1...,Финансовый аналитик,"Москва, м. Крылатское, не готова к переезду, н...",Опыт работы 8 лет 11 месяцев,"Уверенный пользователь ПК: MS Office (Word, Ex...",Высшее образование\n2009\nЧувашский государств...,"Специализации:\nФинансовый аналитик, инвестици...",---,Чувашский государственный университет имени И....,NaN,...,1,"ООО «Рольф» (розничная торговля автомобилями, ...",ООО «ЭлекКом Логистик» (комплексные поставки э...,False,ENG,found smt by rule (Analyst)|(Аналитик данных)|...,['---'],не готова к командировкам,не готова к переезду,Москва
2996,https://hh.ru/resume/0ed518390003eb7c860039ed1...,Аналитик,"Москва, м. Ховрино, не готов к переезду, готов...

In [9]:
list(df)

['Ссылка',
 'Название',
 'Коммандировка',
 'Опыт',
 'О себе',
 'Образование',
 'Интересы',
 'Навыки',
 'Образование-1',
 'Образование-2',
 'Уровень образование',
 'Кол-во образований',
 'Работа 1',
 'Работа 2',
 'Топовость образования',
 'is_eng?',
 'Справочник по интересам',
 'Навыки списком',
 'Релокация?',
 'Коммандировка?',
 'Город?']

In [11]:
df['Опыт'][2]
opyt_normalized = []
for x in df['Опыт']:
    x=x.removeprefix('Work experience ').removeprefix('Опыт')
    x=x.replace('years', ';')
    x=x.replace('year', ';')
    x=x.replace('года', ';')  # ВСЕГДА д. б. раньше
    x=x.replace('год', ';')
    x=x.replace('лет', ';')
    
    if x.find(';') == -1:
        x = '0;' + x
    x=x.removesuffix('months')
    x=x.removesuffix('month')
    x=x.removesuffix('месяцев')
    x=x.removesuffix('месяц')
    x=x.removesuffix('месяца')
     
    x=x.replace(' ', '')
    if re.search('[а–Я]+|[a–Z]+', x):
        x='---'
    opyt_normalized.append(x)
    
df['Опыт нормализованный'] = opyt_normalized

In [12]:
df

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,...,Работа 1,Работа 2,Топовость образования,is_eng?,Справочник по интересам,Навыки списком,Релокация?,Коммандировка?,Город?,Опыт нормализованный
0,https://hh.ru/resume/a179dc8e00074bc7400039ed1...,Администратор салона красоты,"Москва, м. Молодежная, не готова к переезду, г...",Опыт работы 7 лет 1 месяц,"Уверенный пользователь ПК, Ms Word, Ms Excel, ...",Высшее образование\n2015\nФГБУ ВПО «Российский...,Специализации:\nАдминистратор\nЗанятость: полн...,Коммуникации с клиентами и партнерами/Работа с...,ФГБУ ВПО «Российский экономический университет...,ФГБУ ВПО «Саратовская Государственная Юридичес...,...,Салон красоты SHOKO\nУслуги для населения... П...,Кредитный Потребительский Кооператив «Поволжск...,False,ENG,---,['Коммуникации с клиентами и партнерами/Работа...,готова к редким командировкам,не готова к переезду,Москва,---
1,https://hh.ru/resume/cdaf5cfe00003bcc320039ed1...,Руководитель отдела планирования и управления ...,"Москва, м. Выхино, готова к переезду, готова к...",Опыт работы 17 лет 3 месяца,"Энергична, активна, в поиске новых возможносте...",Высшее образование\n2010\nМосковский Государст...,Специализации:\nНачальник склада\nЗанятость: п...,---,Московский Государственный Университет Экономи...,NaN,...,"АО ТД Перекресток\nМосква, karusel.ru/\nРознич...","Дочки & Сыночки, сеть супермаркетов\nРоссия, d...",False,RU,---,['---'],готова к командировкам,готова к переезду,Москва,---
2,https://hh.ru/resume/fb6ac9c200027770d60039ed1...,Начальник отдела продаж,"Шахты, готова к переезду (Москва, Санкт-Петерб...",Опыт работы 18 лет 4 месяца,"опыт прямых продаж, ведение коммерческих перег...",Высшее образование\n2001\nЮжно-Российский госу...,Специализации:\nРуководитель отдела продаж\nЗа...,---,Южно-Российский государственный университет эк...,NaN,...,Государственное бюджетное учреждение Ростовско...,"Государственное бюджетное учреждение ""Лечебно-...",False,RU,---,['---'],готова к командировкам,готова к переезду,Шахты,---
3,https://hh.ru/resume/e23c95770000b1b95f0039ed1...,"Коммерческий директор, Финансовый директор","Москва, м. Бабушкинская, готов к переезду, гот...",Опыт работы 17 лет 5 месяцев,Коммуникабелен\nУмение работать с возражениями...,Высшее образование\n2006\nБрянский государстве...,Специализации:\nКоммерческий директор (CCO)\nФ...,---,Брянский государственный технический университет,Брянский государственный технический университет,...,"Альфа-Банк (Россия)\nМосква, www.alfabank.ru\n...","Банк ВТБ (ПАО)\nМосква, www.vtbcareer.com\nФин...",False,RU,---,['---'],готов к командировкам,готов к переезду,Москва,---
4,https://hh.ru/resume/9c86655b00073d88700039ed1...,QA/auto QA/Performance QA,"Краснодар, готов к переезду (Москва, Санкт-Пет...",Опыт работы 7 лет 10 месяцев,Дисциплина - умение выполнять поставленные зад...,Среднее специальное образование\n2009\nКраснод...,Специализации:\nТестировщик\nЗанятость: стажир...,---,Краснодарский Государственный Гуманитарно Техн...,NaN,...,"JavaRush\nМосква, javarush.ru/\nИнформационные...","Social Links\nМосква, mtg-bi.com\nИнформационн...",False,ENG,---,['---'],готов к командировкам,готов к переезду,Краснодар,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,https://hh.ru/resume/b74645f3000102b1b90039ed1...,Финансовый аналитик,"Москва, м. Крылатское, не готова к переезду, н...",Опыт работы 8 лет 11 месяцев,"Уверенный пользователь ПК: MS Office (Word, Ex...",Высшее образование\n2009\nЧувашский государств...,"Специализации:\nФинансовый аналитик, инвестици...",---,Чувашский государственный университет имени И....,NaN,...,"ООО «Рольф» (розничная торговля автомобилями, ...",ООО «ЭлекКом Логистик» (комплексные поставки э...,False,ENG,found smt by rule (Analyst)|(Аналитик данных)|...,['---'],не готова к командировкам,не готова к переезду,Москва,---
2996,https://hh.ru/resume/0ed518390003eb7c860039ed1...,Аналитик,"Москва, м. Ховрино, не готов к пере

**Словарь образования:**

В данной функции мы составляем словарь образования для создания юинарного признака: входит/не входит в топ? Извлечение данной сущности будет самым _неидеальным_ (омонимия аббревиатур вузов етц), но даже погрешность тут будет скомпенсирована большим количеством более простых признаков.

In [13]:
edu_list = pd.concat([df['Образование-1'], df['Образование-2']], axis=0).drop_duplicates().reset_index().drop('index', axis=1)
edu_list = list(edu_list[list(edu_list)[0]])
len(edu_list)

ys=[]

for x in edu_list:
    s=str(x)
    l = s.find(',')+1
    y=s[l:].lstrip(' ')
    print(end='')
    ys.append(y)
    
for i in range(len(ys)):
    y = ys[i]
    is_space = re.fullmatch('[А-я]+[ ][А-я ()-]+', y)
    if is_space:
        #print(y)
        1==1
    is_m = re.fullmatch('Mosc.*', y) or re.fullmatch('Моск.*', y) or re.fullmatch('Петерб.*', y) or re.fullmatch('Peters.*', y) or re.fullmatch('СПб.*', y) or re.fullmatch('Новосиб.*', y) or re.fullmatch('Novosib.*', y) or re.fullmatch('Томск.*', y) or re.fullmatch('Tomsk.*', y)
    if not is_space and not is_m:
        ys[i] = 'NO'
#ys

cnt=0
for y in ys:
    if y != 'NO':
        cnt+=1
cnt

import numpy as np
ys = np.array(ys)
ys=np.unique(ys)
ys=list(ys)
len(ys)

"""
Топ-10 IT-вузов России по версии QS (Quacquarelli Symonds)

1. Московский государственный университет им. М. В. Ломоносова;
2. Университет ИТМО;
3. Московский физико-технологический институт (МФТИ);
4. Высшая школа экономики;
5. Санкт-Петербургский политехнический университет Петра Великого; 
6. Московский государственный технический университета (МГТУ) имени Н.Э. Баумана;
7. Санкт-Петербургский политехнический университет Петра Великого; 
8. Новосибирский государственный университет;
9. Московский инженерно-физический институт (МИФИ);
10. Томский политехнический университет.

https://propostuplenie.ru/article/spisok-luchshih-informacionnyh-vuzov-rossii/
"""

def is_top(y):
    if re.search('ВШЭ', y) or re.search('Высшая школа экономики', y) or re.search('HSE', y):
        y='ВШЭ'
    elif re.search('Московский Государственный Университет$', y) or re.search('Московский Государственный Университет им[.  ени] (М.|Л)', y) or re.search('МГУ[^ А-я]|\(МГУ\)', y) or re.search('Moscow state University', y) or re.search('MSU', y) or re.search('МГУ', y):
        y='МГУ'
    elif re.search('ИТМО', y):
        y='ИТМО'
    elif re.search('МФТИ', y) or re.search('Московский физико-технологический институт', y):
        y='Физтех'
        # ЗФТШ при МФТИ
        # - Физтех
    elif re.search('СПБ Политех', y) or re.search('Санкт-Петербургский политехнический университет', y):
        y='Политех СПб'
    elif re.search('Бауман', y):
        y='Бауманка'
    elif re.search('НГУ', y) or re.search('Новосибирский государственный университет', y):
        y='НГУ'
        # НГУ Наталии Нестеровой
    elif re.search('МИФИ', y) or re.search('Московский инженерно-физический институт', y):
        y='МИФИ'
        # Какие-то Техникум ДИТИ НИЯУ МИФИ и прочее...
    elif re.search('ТомГУ', y) or re.search('Томский политехнический университет', y):
        y='ТомГУ'
    else:
        y='NO'
    return y != 'NO'
        
# Дальше завожу колонку: parsed_uni по этому правилу -- 1 если хотя бы 1 уник топовый
# И там заменяю значения и вывожу равно ли МГУ / етц.

xs = []
for x in df.index:
    stroke = df.iloc[x].loc[['Образование-1', 'Образование-2']]
    k2 = False
    if type(stroke[0]) != float:
        k1 = is_top(stroke[0])
    if type(stroke[1]) != float:
        k2 = is_top(stroke[1])
    xs.append(k1 or k2)
    
df['Топовость образования']=xs

In [14]:
df['Топовость образования'].value_counts()

False    2687
True      313
Name: Топовость образования, dtype: int64

**Флаг, что текст на eng:**

Здесь мы определим, написан ли текст на английском языке. Сделаем это по разделу "О себе". Понятно, что если он на английском, то и все резюме на английском, поэтому было выбрано именно это поле. 

Старая версия:

```
df['О себе']
cnt=0

for y in df['О себе']:
    if type(y)!=float:  # not nan
        if re.search('[a–Z]+', y):  # более точно: у английских резюме интересы тоже будут на англ итд, потом подумаю над этим
            cnt+=1
cnt, df.shape[0]-cnt

ts=[]
for x in df['О себе']:
    t = 'NOT ENG, NOT RU'
    if type(x)!=float:  # not nan
        t = re.search('[a–Z]+', x)
        if t:
            t = 'ENG'
        else:
            t = 'RU'
    ts.append(t)
df['is_eng?'] = ts
```

In [15]:
ts = []

for i in range(df.shape[0]):
    if type(df['Город?'][i])!=float and type(df['Релокация?'][i])!=float and type(df['Коммандировка?'][i])!=float:  # not nan\
        t = re.search('[A-z]+', df['Город?'][i]) or re.search('[A-z]+', df['Релокация?'][i]) or re.search('[A-z]+', df['Коммандировка?'][i])
        if t:
            t = 'ENG'
        else:
            t = 'RU'

    # print(x, t)
    else:
        t = '---'
    ts.append(t)
        
df['is_eng?'] = ts

In [16]:
about = df['О себе'].loc[df['is_eng?'] == 'RU']

In [17]:
df = df.loc[df['is_eng?'] == 'RU']

In [18]:
about

0       Уверенный пользователь ПК, Ms Word, Ms Excel, ...
1       Энергична, активна, в поиске новых возможносте...
2       опыт прямых продаж, ведение коммерческих перег...
3       Коммуникабелен\nУмение работать с возражениями...
4       Дисциплина - умение выполнять поставленные зад...
                              ...                        
2995    Уверенный пользователь ПК: MS Office (Word, Ex...
2996    Моя потребность в постоянном улучшении себя на...
2997                                                    .
2998                                                  ---
2999                                                  ---
Name: О себе, Length: 2725, dtype: object

In [19]:
df

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,...,Работа 1,Работа 2,Топовость образования,is_eng?,Справочник по интересам,Навыки списком,Релокация?,Коммандировка?,Город?,Опыт нормализованный
0,https://hh.ru/resume/a179dc8e00074bc7400039ed1...,Администратор салона красоты,"Москва, м. Молодежная, не готова к переезду, г...",Опыт работы 7 лет 1 месяц,"Уверенный пользователь ПК, Ms Word, Ms Excel, ...",Высшее образование\n2015\nФГБУ ВПО «Российский...,Специализации:\nАдминистратор\nЗанятость: полн...,Коммуникации с клиентами и партнерами/Работа с...,ФГБУ ВПО «Российский экономический университет...,ФГБУ ВПО «Саратовская Государственная Юридичес...,...,Салон красоты SHOKO\nУслуги для населения... П...,Кредитный Потребительский Кооператив «Поволжск...,False,RU,---,['Коммуникации с клиентами и партнерами/Работа...,готова к редким командировкам,не готова к переезду,Москва,---
1,https://hh.ru/resume/cdaf5cfe00003bcc320039ed1...,Руководитель отдела планирования и управления ...,"Москва, м. Выхино, готова к переезду, готова к...",Опыт работы 17 лет 3 месяца,"Энергична, активна, в поиске новых возможносте...",Высшее образование\n2010\nМосковский Государст...,Специализации:\nНачальник склада\nЗанятость: п...,---,Московский Государственный Университет Экономи...,NaN,...,"АО ТД Перекресток\nМосква, karusel.ru/\nРознич...","Дочки & Сыночки, сеть супермаркетов\nРоссия, d...",False,RU,---,['---'],готова к командировкам,готова к переезду,Москва,---
2,https://hh.ru/resume/fb6ac9c200027770d60039ed1...,Начальник отдела продаж,"Шахты, готова к переезду (Москва, Санкт-Петерб...",Опыт работы 18 лет 4 месяца,"опыт прямых продаж, ведение коммерческих перег...",Высшее образование\n2001\nЮжно-Российский госу...,Специализации:\nРуководитель отдела продаж\nЗа...,---,Южно-Российский государственный университет эк...,NaN,...,Государственное бюджетное учреждение Ростовско...,"Государственное бюджетное учреждение ""Лечебно-...",False,RU,---,['---'],готова к командировкам,готова к переезду,Шахты,---
3,https://hh.ru/resume/e23c95770000b1b95f0039ed1...,"Коммерческий директор, Финансовый директор","Москва, м. Бабушкинская, готов к переезду, гот...",Опыт работы 17 лет 5 месяцев,Коммуникабелен\nУмение работать с возражениями...,Высшее образование\n2006\nБрянский государстве...,Специализации:\nКоммерческий директор (CCO)\nФ...,---,Брянский государственный технический университет,Брянский государственный технический университет,...,"Альфа-Банк (Россия)\nМосква, www.alfabank.ru\n...","Банк ВТБ (ПАО)\nМосква, www.vtbcareer.com\nФин...",False,RU,---,['---'],готов к командировкам,готов к переезду,Москва,---
4,https://hh.ru/resume/9c86655b00073d88700039ed1...,QA/auto QA/Performance QA,"Краснодар, готов к переезду (Москва, Санкт-Пет...",Опыт работы 7 лет 10 месяцев,Дисциплина - умение выполнять поставленные зад...,Среднее специальное образование\n2009\nКраснод...,Специализации:\nТестировщик\nЗанятость: стажир...,---,Краснодарский Государственный Гуманитарно Техн...,NaN,...,"JavaRush\nМосква, javarush.ru/\nИнформационные...","Social Links\nМосква, mtg-bi.com\nИнформационн...",False,RU,---,['---'],готов к командировкам,готов к переезду,Краснодар,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,https://hh.ru/resume/b74645f3000102b1b90039ed1...,Финансовый аналитик,"Москва, м. Крылатское, не готова к переезду, н...",Опыт работы 8 лет 11 месяцев,"Уверенный пользователь ПК: MS Office (Word, Ex...",Высшее образование\n2009\nЧувашский государств...,"Специализации:\nФинансовый аналитик, инвестици...",---,Чувашский государственный университет имени И....,NaN,...,"ООО «Рольф» (розничная торговля автомобилями, ...",ООО «ЭлекКом Логистик» (комплексные поставки э...,False,RU,found smt by rule (Analyst)|(Аналитик данных)|...,['---'],не готова к командировкам,не готова к переезду,Москва,---
2996,https://hh.ru/resume/0ed518390003eb7c860039ed1...,Аналитик,"Москва, м. Ховрино, не готов к переезд

**Справочник по интересам:**

Есть ли аналитические комбинации слов в интересах? Используем простое правило для простоты.

In [20]:
inter_list = []

for inter in df['Интересы']:
    intr = inter.split('\n')
    if re.search('(Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)', inter):
        inter_list.append('found smt by rule (Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)')
    else:
        inter_list.append('---')
        
df['Справочник по интересам']= inter_list

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/2972301623.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Справочник по интересам']= inter_list


**Извлечение сущностей:**

На сайте hh.ru единое поле для трех объектов сразу. Разъединим их с учетом разных знаяений и того, что они перечислены просто через запятую.

In [24]:
xs1 = []
xs2=[]
xs3=[]
for x in df['Коммандировка']:
    # english
    x1 = re.search('[A-z]+', x)
    x2 = re.search('[a-z ]*willing to relocate', x)
    x3 = re.search('[a-z ]*prepared for [a-z]* business trips', x)
    if x1:
        x1=x1[0]
    if x2:
        x2=x2[0]
    if x3:
        x3=x3[0]
    
    if x1 or x2 or x3:  # if found smt in engish — is it not wrong way to do it?
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)
    else:
        # russian — work in progress
        # Москва, не готова к переезду, готова к редким командировкам
        # Санкт-Петербург, м. Гражданский проспект, готов к переезду (Москва), готов к командировкам
        # Москва, готов к переезду (Нидерланды, Испания, Франция, Италия, Австрия, Германия, Швейцария), готов к командировкам
        # Москва, хочу переехать, готов к командировкам
        # Москва, м. Ясенево, не готов к переезду, готов к редким командировкам
        x1 = re.search('[А-я]+', x)  # is it correct?
        x2 = re.search('[А-я]*[ не]*готов[а]* к переезду', x)
        x3 = re.search('[А-я]*[ не]*готов[а]* к[ а-я]* командировкам', x)  # черновое правило
        if x1:
            x1=x1[0]
        if x2:
            x2=x2[0]
        if x3:
            x3=x3[0]
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)

zs = []
for z in df['Навыки']:
    zzz = z.split('\n')
    zs.append(zzz)
    
df['Навыки списком'] = zs
df['Релокация?'] = xs3
df['Коммандировка?'] = xs2
df['Город?'] = xs1

"""
1. Коммандировка 
2. Навыки
3. Кол-во работ
4. Наличие аналитических комбинаций слов в работе — новый столбец
"""

import numpy as np
import re

#print(df['Коммандировка'][0])
#print(re.search('[A-z]+', df['Коммандировка'][0Z]))
#print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][8]))
#print(re.search('[a-z ]*prepared for business trips', df['Коммандировка'][8]))

#print(df['Коммандировка'][0])
#print(re.search('[A-z]+', df['Коммандировка'][0]))
#print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][0]))
#print(re.search('[a-z ]*prepared for [a-z]* business trips', df['Коммандировка'][0]))

df['Навыки'].iloc[0].split('\n')  # вот так?

work_list = df['Где работал?']
analytics_words = []
for work in work_list:
    # пока супер примитивное правило:
    if re.search('(SQL)|(Excel)|(Python)|(Tableau)|(Power BI)', work):
        analytics_words.append('found smt by rule (SQL)|(Excel)|(Python)|(Tableau)|(Power BI)')
    else:
        analytics_words.append('---')
df['Аналитические комбинации слов'] = analytics_words

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/111569273.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Навыки списком'] = zs
/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/111569273.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Релокация?'] = xs3
/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/111569273.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

KeyError: 'Где работал?'

**Перевод уровня образования:**

Определим уровень образования (высшее, ... — какие возможные значения?). Мы увидели разные стили написания. Составить в Markdown таблицу перевода.

In [25]:
z = list(np.unique(list(df['Уровень образование'])))

for y in z:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    print(y)
    
ys=[]
for y in df['Уровень образование']:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    ys.append(y)
df['Уровень оборазования нормализованный'] = ys

Высшее образование
Бакалавр
Доктор наук
Кандидат наук
Магистр
Неоконченное высшее образование
Образование
Среднее образование
Среднее специальное образование


/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/1620052491.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Уровень оборазования нормализованный'] = ys


**Сохранение результатов:**

In [26]:
list(df)[:6]

['Ссылка', 'Название', 'Коммандировка', 'Опыт', 'О себе', 'Образование']

**Теперь немного про карьерный рост:**

```
#df['Работа']
list(df)
```

для списка карьерного роста надо будет собрать список компаний где был и найти там совпадения, на текущих данных так сделать нельзя

In [43]:
driver = webdriver.Chrome('./chromedriver')

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/2299280398.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


Добавим еще один признак — карьерный рост: есть ли повторения в опыте работы? (Тоже наверх это)

In [44]:
repet = []

In [45]:
for i in range(df.shape[0]):
    driver.get(df.iloc[i]['Ссылка'])
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    w = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]/div[@class="bloko-text bloko-text_strong"][not(@data-qa="resume-block-experience-position")]//span')
    
    works = []
    for ww in w:
        try:
            works.append(ww.text)
        except:
            1==1
    
    works_normilized = []
    for w in works:
        works_normilized.append(morph.normal_forms(w)[0])
    
    repet.append(len(set(works_normilized)) - len(works_normilized) < 0)

проверить, правда ли так много людей с повышениями:

In [46]:
repet[:3]

[False, False, False]

теперь для нормализованных названий найти повторения: `len(set(works)) - len(works) > 0` — и чекнуть адекватность такого определения

^ почти готово

**Для экономии времени — ведь этот признак был придуман позже**

In [47]:
df['Карьерный рост'] = repet

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/1520391026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Карьерный рост'] = repet


_куда-то надо сохранить этот новый датасет в приличное место (или вообще туда же, просто старую версию датасета переместить в backup)_:

In [48]:
# dataset.save()

а также я давно хотел заняться топовостью обоазования

вообще там вроде все ок, тогда посмотрю вручную при разметке есть ли проблемы и почему, составлю список прямо

In [49]:
# df=df.drop(list(df)[0], axis=1)

Сохраним полученные результаты:

In [50]:
df.to_csv('resumes_features.csv')

### Что дальше? (Раньше было частью черновика)

In [ ]:
df = pd.read_csv('resumes_features.csv')

Осталось доделать справочник по образованию еще лучше, а также справочник по интересам и наличие аналитчиеских комбинаций слов (см черновик, что сделать)

**начинаем АД:**

ключевые совпадения см л34 она скинула

**Извлечение сущностей: более совершенные правила.** ВКР. Если город и коммандировка на английском то резюме точно на английском. Какие-то такие правила еще поискать. Посмотрю только в каком таком точном поле меньше всего пропусков. ВКР: позже вычленяю ещё сущности как мы обсуждали см prev call в 7.12. См github: допилить еще справочник образования и прочее, чтобы все в этом плане было идеально

В `scraping.ipynb` были собраны и подготовлены данные. Возможно, стоит еще доработать (в разделе "пост-обработка"):
- Словарь образования
- Справочник по интересам
- Аналитические комбинации слов

#### Предобработка полученных данных для анализа

**mBART перед построением токенов в "О себе"**

Описать как это работает. Вставить это перед построением эмбеддингов!!

https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta?text=%D0%97%D0%B0+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D0%B8+%D1%83%D1%87%D1%91%D0%B1%D1%8B%2C+%D1%8F+%D0%BE%D0%B2%D0%BB%D0%B0%D0%B4%D0%B5%D0%BB+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D1%8F%D0%BC%D0%B8%2C+%D0%BD%D0%B5%D0%BE%D0%B1%D1%85%D0%BE%D0%B4%D0%B8%D0%BC%D1%8B%D0%BC%D0%B8+%D0%B4%D0%BB%D1%8F+%D0%B8%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D0%B8+%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%BE%D0%B3%D0%BE+%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D1%8B%D1%85+%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B9+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F%2C+%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0+%D0%B8+%D1%81%D0%B8%D0%BD%D1%82%D0%B5%D0%B7%D0%B0+%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D1%85+%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B9+%D0%B2+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D0%B0%D1%85+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D0%B9+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%86%D0%B8%D0%B8.%0A%D0%AF+%D0%BE%D0%B1%D0%BB%D0%B0%D0%B4%D0%B0%D1%8E+%D1%88%D0%B8%D1%80%D0%BE%D0%BA%D0%B8%D0%BC+%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D0%BE%D0%BC+%D0%B2+%D0%BF%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B8+BI+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8%2C+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8%2C+%D1%81%D0%B2%D1%8F%D0%B7%D1%8F%D1%85%2C+%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85+%D0%B2+Power+Qwery+%D0%B8+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D1%81+%D0%B1%D0%B0%D0%B7%D0%B0%D0%BC%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.%0A%D0%A3%D1%87%D0%B0%D1%81%D1%82%D0%B2%D0%BE%D0%B2%D0%B0%D0%BB+%D0%B2+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B0%D1%85+%D0%BF%D0%BE+%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B8+%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81-%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%BE%D0%B2+%D0%B8+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8.%0A%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BB+%D0%B2+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D1%8F%D1%85+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Power+BI%2C+Excel%2C+MS+Office%2C+VipPlanner.%0A%D0%97%D0%BD%D0%B0%D0%BA%D0%BE%D0%BC+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%BD%D1%8B%D0%BC%D0%B8+%D0%B8%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Jira%2C+kanban%2C+confluence.%0A%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B+%D0%B4%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4%D0%BE%D0%B2+%D0%BC%D0%BE%D0%B3%D1%83+%D0%B2%D1%8B%D1%81%D0%BB%D0%B0%D1%82%D1%8C+%D0%BB%D0%B8%D1%87%D0%BD%D0%BE.%0A%D0%91%D1%83%D0%B4%D1%83+%D1%80%D0%B0%D0%B4+%D0%B2+%D1%80%D0%B0%D0%BC%D0%BA%D0%B0%D1%85+%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%B8%D1%82%D1%8C+%D0%BD%D0%B0+%D0%B2%D1%81%D0%B5+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D1%83%D1%8E%D1%89%D0%B8%D0%B5+%D0%92%D0%B0%D1%81+%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D1%8B.%0A%D0%97%D0%B0%D1%80%D0%B0%D0%BD%D0%B5%D0%B5+%D0%B1%D0%BB%D0%B0%D0%B3%D0%BE%D0%B4%D0%B0%D1%80%D1%8E+%D0%92%D0%B0%D1%81+%D0%B7%D0%B0+%D1%83%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D0%BE%D0%B5+%D0%BC%D0%BE%D0%B5%D0%B9+%D0%BA%D0%B0%D0%BD%D0%B4%D0%B8%D0%B4%D0%B0%D1%82%D1%83%D1%80%D0%B5+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F.%0A%D0%92%D0%BB%D0%B0%D0%B4%D0%B5%D1%8E+%D0%BD%D0%B0%D0%B2%D1%8B%D0%BA%D0%BE%D0%BC+%22%D0%94%D0%B5%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F+%D0%BF%D0%B5%D1%80%D0%B5%

попробуем:

In [52]:
print(df['О себе'][14])

Компьютерные навыки: MS Office 2007, Internet. Профессиональные навыки: умение четко организовывать работу, своевременно принимать решения, легко обучаем, открыт новым проектам, умение вести переговоры и отстаивать интересы компании, ответственность, исполнительность, коммуникабельность, инициативность, четкость, аккуратность, высокая работоспособность.


In [53]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [54]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

^ ок, снизу ... просто тест

In [55]:
def summarize(article_text):
    input_ids = tokenizer(
        [article_text],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

Как работает функция:

In [60]:
df['О себе'].iloc[0]

'Уверенный пользователь ПК, Ms Word, Ms Excel, 1С, Power Point, Internet Explorer, amoCRM, Bitrix24 и офисной техники;\nПравовых систем Гарант и Консультант +;\nЗнание законодательства;\nРабота с входящей и исходящей корреспонденцией,\nЭтика делового общения, общения с клиентами;\nНавыки работы по взаимодействию с государственными органами и общественными организациями;\nНеоднократное участие в судебных заседаниях;\nАнализ данных, составление договоров, запросов, отзывов, заявлений, уведомлений,\nКонсультирование клиентов, регистрация договоров,\nОрганизация мероприятий,\nРешение конфликтных ситуаций,\nНавыки общения с людьми,\nУмение работать в команде и.др.\n\nЗнание языков: Английский – средний.\nЛичные качества: По характеру общительная, ответственная, коммуникабельная, внимательная, активная, коммуникативная, эрудированная, креативная, доброжелательная, уравновешенная, инициативная, способна быстро обучаться, имею организаторские навыки, а так же интерес к дизайну, маркетингу, выс

In [61]:
summarize(df['О себе'].iloc[0])

'Уверенный пользователь ПК, Ms Word, Ms Excel, 1С, Power Point, Internet Explorer, amoCRM, Bitrix24 и офисной техники; Правовых систем Гарант и Консультант +; Знание законодательства; Работа с входящей и исходящей корреспонденцией, Этика делового общения, общения с клиентами; Навыки работы по взаимодействию с государственными органами и общественными организациями; Неоднократное участие в судебных заседаниях; Анализ данных, составление договоров, запросов, отзывов, заявлений, уведомлений.'

In [62]:
summarizations = []
for i in range(df.shape[0]):
    if df['О себе'].iloc[i] != '---' and df['is_eng?'].iloc[i] == 'RU' and (not pd.isna(df['О себе'].iloc[i])):
        text_to_summarize = df['О себе'].iloc[i]
        summarizations.append(summarize(text_to_summarize))
    else:
        summarizations.append(df['О себе'].iloc[i])
    
    if len(summarizations) % 5 == 0:
        print(len(summarizations), '/', df.shape[0])

5 / 2725
10 / 2725
15 / 2725
20 / 2725
25 / 2725
30 / 2725
35 / 2725
40 / 2725
45 / 2725
50 / 2725
55 / 2725
60 / 2725
65 / 2725
70 / 2725
75 / 2725
80 / 2725
85 / 2725
90 / 2725
95 / 2725
100 / 2725
105 / 2725
110 / 2725
115 / 2725
120 / 2725
125 / 2725
130 / 2725
135 / 2725
140 / 2725
145 / 2725
150 / 2725
155 / 2725
160 / 2725
165 / 2725
170 / 2725
175 / 2725
180 / 2725
185 / 2725
190 / 2725
195 / 2725
200 / 2725
205 / 2725
210 / 2725
215 / 2725
220 / 2725
225 / 2725
230 / 2725
235 / 2725
240 / 2725
245 / 2725
250 / 2725
255 / 2725
260 / 2725
265 / 2725
270 / 2725
275 / 2725
280 / 2725
285 / 2725
290 / 2725
295 / 2725
300 / 2725
305 / 2725
310 / 2725
315 / 2725
320 / 2725
325 / 2725
330 / 2725
335 / 2725
340 / 2725
345 / 2725
350 / 2725
355 / 2725
360 / 2725
365 / 2725
370 / 2725
375 / 2725
380 / 2725
385 / 2725
390 / 2725
395 / 2725
400 / 2725
405 / 2725
410 / 2725
415 / 2725
420 / 2725
425 / 2725
430 / 2725
435 / 2725
440 / 2725
445 / 2725
450 / 2725
455 / 2725
460 / 2725
465 / 27

23:35 20

23:37 25

0:07 100

2:12 420

3:43 650

13:00 2125

16:10 2630

In [63]:
df['О себе, summarized'] = summarizations

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/1096740388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['О себе, summarized'] = summarizations


In [66]:
df.to_csv('resumes_features.csv')

Дальше: article_text = `df['О себе'][0]`; мб надо будет по частям обрабатывать ибо это будет долго и затратно по памяти чтобы ничего не упало и не испортило всю работу

Итак, мы извлекли сущности и привели данные к деиному формату. Теперь займемся дальнейшей предобработкой (этапом): токенизация.

Мы будем осущесвлять ее с помощью следующего алгоритма. Берем ...